In [2]:
#packages
import pandas as pd
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
# loading data
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
train_df.head()
test_df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [26]:
print(len(train_df))
print(len(test_df))

1460
1459


In [5]:
# getting ready x and y variables
x = train_df.drop(['Id','SalePrice'],axis=1)
y = train_df.SalePrice

# train test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 42, )

In [27]:
y_train.sort_index()

0       208500
1       181500
2       223500
3       140000
4       250000
         ...  
1455    175000
1456    210000
1457    266500
1458    142125
1459    147500
Name: SalePrice, Length: 978, dtype: int64

In [8]:
#setting up the model
model = lgb.LGBMClassifier(learning_rate = 0.09, max_depth = -5, random_state = 42)
model.fit(x_train, y_train, eval_set = [(x_test, y_test), (x_train, y_train)],
          verbose = 20, eval_metric = 'logloss')



ValueError: y contains previously unseen labels: [35311, 40000, 55993, 62383, 64500, 66500, 67000, 68400, 68500, 75000, 75500, 79500, 84900, 85400, 87500, 89471, 90350, 91300, 92000, 93500, 94000, 101800, 102776, 103200, 104000, 105500, 105900, 107900, 108480, 108959, 109008, 116050, 118400, 118858, 121500, 126175, 126500, 128200, 128950, 130250, 131400, 133500, 134432, 134500, 135500, 135750, 135900, 139900, 144500, 149350, 149500, 150900, 151500, 153337, 153575, 154300, 154500, 154900, 155835, 155900, 156500, 159950, 161500, 164700, 165600, 169990, 172400, 173500, 178900, 181134, 182000, 183500, 183900, 184900, 185750, 185900, 187100, 187750, 192140, 194201, 195400, 200624, 204900, 205950, 206300, 215200, 216000, 217500, 219210, 233170, 238000, 239500, 243000, 244400, 245350, 246578, 253000, 253293, 254000, 254900, 255000, 262000, 263000, 264132, 271900, 274900, 276000, 283463, 284000, 293077, 297000, 301000, 305000, 310000, 311500, 311872, 313000, 314813, 315500, 317000, 318061, 337

In [6]:
#Accuracy
print('Training accuracy {:.4f}'.format(model.score(x_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(x_test,y_test)))

In [ ]:
#Relevant variables
lgb.plot_importance(model)

In [ ]:
# Learning curve of the model
lgb.plot_metric(model)


In [ ]:
# plot tree of the model
lgb.plot_tree(model,figsize=(30,40))

In [ ]:
# confusion matrix
metrics.plot_confusion_matrix(model,x_test,y_test,cmap='Blues_r')

In [ ]:
# metrics
print(metrics.classification_report(y_test,model.predict(x_test)))

In [ ]:
#predict and getting data out from h2o
predictions = 

In [15]:
#merging predictions with the full data set
submission = pd.merge(test_df['Id'],
                          predictions,
                          left_index = True,
                          right_index = True)
submission.rename(columns = {'predict': 'SalePrice'}, inplace = True)

In [16]:
# save to csv
submission.to_csv('../data/submission_' + best_model_name + '.csv',
                  header = True, index = False)

# 00. Score of 0.12868
# 01. Change from 25 max models to 1000. Score of 0.12792
# 02. Sorting leaderboard by rmse as the main metric on h2o training. Score of 0.12792 made the same.
# 02. Gonna make models for 30 min instead doing 1000 models. No improvement 0.13452
# 02. We're going with 10.000 models. Score of 0.12970... no improving
# 03. Trying with xgboost